## Run Transformer models on hand-labeled policy field classification tweets

In [10]:
import pandas as pd
import os
import re
from dotenv import load_dotenv
from openai import OpenAI

In [11]:
load_dotenv()
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY")) # get API key from .env file

In [12]:
# set a seed for sampling
SEED = 1349

In [13]:
# read data
data = pd.read_csv("CAP_sample_coded.tsv", 
                   sep="\t", 
                     dtype={"_id": str} # _id column as string to avoid scientific notation
                     )
data
                   

,_id,verkehr,entwicklung,gesellschaft,innere_sicherheit,verteidigung,aeusseres,digitalisierung_technik,soziales,umwelt,...,kultur_medien_sport,gesundheit,landwirtschaft_ernaehrung,bildung_forschung,wirtschaft,none,intercoder_sample,coder,wirtschaft_finanzen_merge,_source.text
0,922866835887984641,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,1,False,"Der ""schwarze Block"" hat übrigens heute (links..."
1,923412288308236288,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,1,False,VW-Chef Müller fordert Superministerium Wirtsc...
2,923452548442460160,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,1,False,300.000 Polizisten sorgen Tag für Tag für unse...
3,926501188236869634,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,1,False,"~ 1️⃣2️⃣,5️⃣ Mio. Deutsche haben per Zweitstim..."
4,927635670352441346,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,1,False,"#AfD-Doppelmandatsträger:\r\nSpangenberg, Mros..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2394,1522118401937793033,False,False,False,True,False,False,False,False,False,...,False,False,False,False,True,False,True,0,True,Eil: Gericht erklärt #Wirecard-Bilanzen für ni...
2395,1567473763280719872,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,True,0,True,"Fraktionsvize @mueller_sepp stellt fest, für O..."
2396,1569284933209710594,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,True,0,False,Ehren Flyeralarm :) https://t.co/5BPNl1VSeu
2397,1633203373263527940,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,True,0,False,RT @HarjitSajjan: Very productive meeting with...


In [14]:
# we remove the categories "wirtschaft" (economy) and "haushalt_finanzen" (finance) in favor of the compound category "wirtschaft_finanzen"
data = data.drop(columns=["wirtschaft", "haushalt_finanzen"])
# rename "wirtschaft_finanzen_merge" to "wirtschaft_finanzen"
data = data.rename(columns={"wirtschaft_finanzen_merge": "wirtschaft_finanzen"})
data

,_id,verkehr,entwicklung,gesellschaft,innere_sicherheit,verteidigung,aeusseres,digitalisierung_technik,soziales,umwelt,...,arbeit,kultur_medien_sport,gesundheit,landwirtschaft_ernaehrung,bildung_forschung,none,intercoder_sample,coder,wirtschaft_finanzen,_source.text
0,922866835887984641,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,1,False,"Der ""schwarze Block"" hat übrigens heute (links..."
1,923412288308236288,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,1,False,VW-Chef Müller fordert Superministerium Wirtsc...
2,923452548442460160,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,1,False,300.000 Polizisten sorgen Tag für Tag für unse...
3,926501188236869634,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,1,False,"~ 1️⃣2️⃣,5️⃣ Mio. Deutsche haben per Zweitstim..."
4,927635670352441346,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,1,False,"#AfD-Doppelmandatsträger:\r\nSpangenberg, Mros..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2394,1522118401937793033,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,True,0,True,Eil: Gericht erklärt #Wirecard-Bilanzen für ni...
2395,1567473763280719872,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,0,True,"Fraktionsvize @mueller_sepp stellt fest, für O..."
2396,1569284933209710594,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,True,0,False,Ehren Flyeralarm :) https://t.co/5BPNl1VSeu
2397,1633203373263527940,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,True,0,False,RT @HarjitSajjan: Very productive meeting with...


In [15]:
# pivot data to long format
data_long = data.melt(id_vars=["_id", "intercoder_sample", "coder", "_source.text"], 
                      var_name="label")

data_long = data_long[data_long.value == True] # only keep tweets labeled as a certain category (drop False values)

# remove value and coder column
data_long = data_long.drop(columns=["value", "coder"])

data_long



,_id,intercoder_sample,_source.text,label
802,1623756907532910593,False,Das #Deutschlandticket ist ein richtiger Schri...,verkehr
806,1400799340021616640,False,Das Jobticket ist nun seit einem Monat am Star...,verkehr
825,1044258236663304193,False,Der Antrag zu einem Sozialticket für Bus und B...,verkehr
853,1483489042691940357,False,Die Stadt Leverkusen inkl. Oberbürgermeister u...,verkehr
866,1109875138516959234,False,Diese Kampagne bringt wohl mehr als die üblich...,verkehr
...,...,...,...,...
40770,1360263802294579201,True,Der #Unternehmerlohn für Kulturschaffende und ...,wirtschaft_finanzen
40774,1457645827078434818,True,"Schätzungen gehen davon aus, dass jedes Jahr m...",wirtschaft_finanzen
40778,1522118401937793033,True,Eil: Gericht erklärt #Wirecard-Bilanzen für ni...,wirtschaft_finanzen
40779,1567473763280719872,True,"Fraktionsvize @mueller_sepp stellt fest, für O...",wirtschaft_finanzen


In [16]:
# nr of unique documents that are labeled. Note: not all document in the data had been labeled, hence the reduced number!
data_long._id.nunique()

597

In [17]:
# get duplicated rows by _id (i.e. documents fitting multiple labels)
multi_label = data_long[data_long.duplicated('_id', keep = False)]
multi_label._id.nunique() # 181 out of 597 documents have more than one label

181

In [18]:
 # use multilabel data for testing (keep only ID and text)
sample = multi_label.drop_duplicates(subset="_id", keep="first")[['_id', '_source.text']]
sample

,_id,_source.text
806,1400799340021616640,Das Jobticket ist nun seit einem Monat am Star...
825,1044258236663304193,Der Antrag zu einem Sozialticket für Bus und B...
884,1295351591890018306,Es geht um #KfZZulassung. https://t.co/DClf0npcfu
894,1148476943991484416,"Fahrpreiserhöhungen sind nie sexy, aber das #T..."
913,956834625028022274,Gesunder Menschenverstand hat sich durchgesetz...
...,...,...
29712,1371897354635329540,Gute Insights heute @GRUENE_LSA mit @Connylue ...
30986,1253224498171650050,RT @WolfgangWiehle: Linksgrüne Abbruchfantasie...
34546,1572559296621350914,Hier ist jetzt auch ganz offiziell die Nicht-A...
35727,1032945820222582787,Dampfturbinen werden zur Stromgewinnung klassi...


In [19]:
# take a random sample of 10 documents as test input
test_input = sample.sample(10, random_state=SEED)

In [ ]:
# dictionaries for labels in English and German
labels_en = {
'verkehr':'Transportation', 
'entwicklung':'Foreign Aid', 
'gesellschaft':'Civil Rights, Minority Issues and Civil Liberties', 
'innere_sicherheit':'Domestic Policy',
'verteidigung':'Defence Policy', 
'aeusseres':'Foreign Policy', 
'digitalisierung_technik':'Digitalisation, Science and Technology', 
'soziales':'Social Welfare',
'umwelt':'Environmental Policy', 
'europa':'European Policy', 
'arbeit':'Labour, Employment and Immigration', 
'kultur_medien_sport':'Culture, Media and Sports', 
'gesundheit':'Healthcare',
'landwirtschaft_ernaehrung':'Agriculture', 
'bildung_forschung':'Education and Research', 
'none':'none',
'wirtschaft_finanzen':'Economy, Energy and Finance',
}

labels_de = {
'verkehr':'Transport und Verkehr', 
'entwicklung':'Entwicklungszusammenarbeit', 
'gesellschaft':'Bürger- und Freiheitsrechte, Minderheitsrechte', 
'innere_sicherheit':'Innenpolitik und Sicherheit',
'verteidigung':'Verteidigungspolitik', 
'aeusseres':'Außenpolitik', 
'digitalisierung_technik':'Digitalisierung, Wissenschaft und Technologie', 
'soziales':'Sozialpolitik',
'umwelt':'Umweltpolitik', 
'europa':'Europapolitik', 
'arbeit':'Arbeitwesen und Arbeitsmigration', 
'kultur_medien_sport':'Kultur, Medien und Sport', 
'gesundheit':'Gesundheitswesen',
'landwirtschaft_ernaehrung':'Landwirtschaft', 
'bildung_forschung':'Bildung und Forschung', 
'none':'keins',
'wirtschaft_finanzen':'Volkswirtschaft, Energie und Bankwesen',
}

In [21]:
# names of columns in "data"
data.columns

Index(['_id', 'verkehr', 'entwicklung', 'gesellschaft', 'innere_sicherheit',
       'verteidigung', 'aeusseres', 'digitalisierung_technik', 'soziales',
       'umwelt', 'europa', 'arbeit', 'kultur_medien_sport', 'gesundheit',
       'landwirtschaft_ernaehrung', 'bildung_forschung', 'none',
       'intercoder_sample', 'coder', 'wirtschaft_finanzen', '_source.text'],
      dtype='object')

In [ ]:
# instructions (english)

